In [ ]:
%pylab inline

In [ ]:
import numpy
import pandas

In [ ]:
hist_kw = dict(bins=60, normed=True, alpha=0.5)

# (3 points) KS investigation (with negative weights)

Download 

* `check_agreement.csv`, 
* `training.csv`

to the folder `datasets/` from https://www.kaggle.com/c/flavours-of-physics/data

`check_agreement.csv` is a control channel, where 1-label corresponds to MC, 0-label - real data, `weight` - sPlot weights

In [ ]:
data_agreement = pandas.read_csv('datasets/check_agreement.csv')
features_for_ks = ['LifeTime', 'FlightDistance', 'IPSig', 'SPDhits']

In [ ]:
for feature in features_for_ks:
    hist(data_agreement[data_agreement.signal == 1][feature].values, label='MC', **hist_kw)
    hist(data_agreement[data_agreement.signal == 0][feature].values, label='real', **hist_kw)
    legend()
    title(feature)
    show()

### Check how sPlot weights change real data pdfs

In [ ]:
# Plot the same histograms with sPlot weights

## Write KS for weights pdfs
standard function doesn't support weights :(

In [ ]:
from sklearn.metrics import roc_curve
def compute_ks(pdf1, pdf2, weights1=None, weights2=None):
    # Write KS metric between two weighted pdfs (see slides)
    ...

## KS metric pdf generation

There are several things to know before doing it:

* we cannot produce two samples from the same distribution with the weights, thus if we generate two samples from uniform and then compute weighted ks metric, obtained value will be greater than it is. That is why during testing hypothesis the probability of error type I will be greater than it is really (Our KS pdf will give the upper estimation of probability to reject H0, when it is true)

* the second sample contains signal and background, which are sampled from different distributions.  
In the second sample we need to generate not only signal (from the same distribution), but also bck from any distribution. But if we remember about sPlot weights property (they compensate bck) then we can conclude that generated bck could be compensated in sample using weights. Of course, some fluctuations can arise (maybe we should somehow estimate them)!

* in the first approximation we can generate two samples as before

### TODO

* Check that for unweighted samples KS will be lower than for weighted.

In [ ]:
# finish the function
def generate_ks_pdf(n1, n2, w1=None, w2=None, points=3000):
    ks = []
    # for each point 
    for step in range(points):
        # generate pdf1 and pdf2 from the same distributions
        ...
        # compute ks for generated pdfs
        ...
    return numpy.array(ks)

In [ ]:
n1 = numpy.sum(data_agreement.signal == 0)
n2 = numpy.sum(data_agreement.signal == 1)
w1 = data_agreement[data_agreement.signal == 0]['weight'].values
w2 = data_agreement[data_agreement.signal == 1]['weight'].values
ks_pdf = generate_ks_pdf(n1, n2)
ks_pdf_weight = generate_ks_pdf(n1, n2, w1=w1, w2=w2)

In [ ]:
hist(ks_pdf, label='simple', **hist_kw)
hist(ks_pdf_weight, label='weight', **hist_kw)
legend()
pass

### TODO:
What features have least disagreement among `features_for_ks`?

# (3 points) Agreement restriction on the classifiers

How more disagreement features can influence the predictions agreement and classifier's quality?

Do for this:

* train any ensemble model on all features
* remove `SPDhits` from training

In the competition your ks metric should be less than **0.09**

In [ ]:
data = pandas.read_csv('datasets/training.csv')
train_features = list(set(data_agreement.columns) - {'id', 'signal', 'weight'})
train_features_wo_spd = list(set(data_agreement.columns) - {'id', 'signal', 'weight', 'SPDhits'})

In [ ]:
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [ ]:
# Divide train on train, test
train_index, test_index = train_test_split(range(len(data)))
train = data.iloc[train_index, :]
test = data.iloc[test_index, :]

In [ ]:
# define function to test model on ks and calculate quality
def test_model(model, features):
    probs = model.predict_proba(data_agreement[features])[:, 1]
    pdf1 = probs[data_agreement.signal.values == 0]
    pdf2 = probs[data_agreement.signal.values == 1]
    model_agr = compute_ks(pdf1, pdf2, weights1=w1, weights2=w2)
    print 'Agreement', model_agr, model_agr < 0.09
    print 'AUC', roc_auc_score(test.signal.values, model.predict_proba(test[features])[:, 1])

### Does subsample parameter influence the agreement and the quality?

### Does learning rate influence?

### Summarize:

* The simple solution to improve an agreement is to remove too disagreement features from the training. 
* There are exist smart solutions which will be study on the next seminars.

## (8 points) Analysis circle 

This exercise will be used at reproducible reserach seminar, thus you necessary should do it.

TODO
* train classifier to discriminate signal and background
* predict control channel (`check_agreement.csv`)
* intoduce correction factor $\alpha(threshold)$ for signal efficiency in signal channel using cotrol channel
* optimize threhsold choice maximizing $\alpha^2 TPR^2 / FPR$ (use holdout)
* compute corrected signal efficiency ($\alpha * TPR$) for signal channel and background efficiency ($FPR$) for chosen threshold 
* compute efficiencies for MC signal and MC control for chosen threshold. Their difference is a systematic error on efficiency.